In [ ]:
%reload_ext autoreload
%autoreload 2

import torch

torch.manual_seed(42)


In [ ]:
from data.shakespeare_data_source import ShakespeareDataSource
from data.tokenizer import Tokenizer
from learning.shakespeare_generator.shakespeare_dataset import ShakespeareDataset


tokenizer = Tokenizer()

shakespeare_data_source = ShakespeareDataSource.load(
    file_path="../datasets/shakespeare/input.txt",
    tokenizer=tokenizer,
)

shakespeare_dataset = ShakespeareDataset(
    shakespeare_data_source=shakespeare_data_source,
    tokenizer=tokenizer,
    sequence_length=8,
)

print(shakespeare_dataset[0])


In [ ]:
from learning.shakespeare_generator.model import ShakespeareGenerator


test_model = ShakespeareGenerator(
    input_size=tokenizer.vocab_size,
    padding_idx=tokenizer.pad_token_idx,
)

# shape: [2, 1] -- (B=2, S=1)
inputs = torch.zeros((2, 1), dtype=torch.long)
outputs = test_model.generate(
    inputs,
    max_length=100,
)

print("---------- Predict from empty input:")
for i in range(outputs.shape[0]):
    print(tokenizer.i2t(outputs[i].tolist()))

# shape: [1, S]
inputs = shakespeare_dataset[0].input.unsqueeze(0)
outputs = test_model.generate(
    inputs,
    max_length=100,
)

print("---------- Predict from first sample:")
for i in range(outputs.shape[0]):
    print(tokenizer.i2t(outputs[i].tolist()))


In [ ]:
from learning.shakespeare_generator.model import Batch, ParallelBatchLearner
from torch.utils.data import DataLoader

from torch import optim
from torch import nn

BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.001
PATIENCE = None
MIN_DELTA = None

model = ShakespeareGenerator(
    input_size=tokenizer.vocab_size,
    padding_idx=tokenizer.pad_token_idx,
)

criterion = nn.CrossEntropyLoss(reduction="sum")
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

learner = ParallelBatchLearner(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
)

train_dataloader = DataLoader(
    shakespeare_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=Batch.from_samples,
)

learner.train(
    dataloader=train_dataloader,
)

In [ ]:
# shape: [2, 1] -- (B=2, S=1)
inputs = torch.zeros((2, 1), dtype=torch.long)
outputs = model.generate(
    inputs,
    max_length=100,
)

print("---------- Predict from empty input:")
for i in range(outputs.shape[0]):
    print(tokenizer.i2t(outputs[i].tolist()))
